In [64]:
import boto3
import json

# Create a Boto3 client for Bedrock Runtime
bedrock_client = boto3.client(service_name='bedrock-runtime')

# Define the prompt and other parameters
prompt = "\n\nHuman: story of two dogs\n\nAssistant:"
modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

# Invoke the model
response = bedrock_client.invoke_model(
    modelId=modelId,
    accept=accept,
    contentType=contentType,
    body=json.dumps({"prompt": prompt, "max_tokens_to_sample": 300, "temperature": 0.1, "top_p": 0.9})
)

# Process the response
response_body = json.loads(response.get('body').read())
print(response_body.get('completion'))


 Here is a story about two dogs:

Rex and Charlie were best friends who did everything together. They lived next door to each other with their human families and spent all day playing in the backyard. Rex was a golden retriever, always happy and eager for fun. Charlie was a German shepherd, more serious but very loyal. 

Every morning, Rex and Charlie would wake up and meet at the fence between their yards. They would bark and wag their tails, excited to start a new day of adventure. First, they would race around the yard, chasing each other back and forth. Then they would grab a toy and play tug-of-war, growling playfully. After getting all their energy out, they would flop down under the big oak tree to nap in the shade. 

When it was time for dinner, Rex and Charlie would eat quickly so they could go back outside. They loved exploring the woods behind their houses, sniffing for new smells and chasing critters. One day, they even came across a pond and spent all afternoon splashing a

In [65]:
import logging

import boto3
import json

# Create a Boto3 client for Bedrock Runtime
bedrock_client = boto3.client(service_name='bedrock-runtime')

max_tokens_to_sample = 200

# Define the prompt and other parameters
prompt = f"""\n\nHuman: 
Write a long and high-quality story about two dogs. Make the story longer than {max_tokens_to_sample}

Rex and Charlie were best friends who did everything together. They lived next door to each other with their human families and spent all day playing in the backyard. Rex was a golden retriever, always happy and eager for fun. Charlie was a German shepherd, more serious but very loyal. 

Every morning, Rex and Charlie would wake up and bark excitedly, ready to start the day's adventures. Their families would let them out into the backyard and they'd run around chasing each other and sniffing for interesting smells. After tiring themselves out, they'd nap in the shade of the big oak tree, Rex's tail still thumping contentedly even in his sleep. 

\n\nAssistant:"""



modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

body = json.dumps({
    "prompt": prompt,
    # specify the parameters as needed
    "max_tokens_to_sample": max_tokens_to_sample,
    "temperature": 0.1,
    "top_p": 0.9,
})

In [66]:
response = bedrock_client.invoke_model(
    modelId=modelId,
    accept=accept,
    contentType=contentType,
    body=body
)
# Process the response
response_body = json.loads(response.get('body').read())
logging.info(response_body.get('completion'))

### Testing the throughput and lantency with locust

In [51]:
!pip install locust

In [68]:
%%writefile locustfile.py

from locust import User, task, between
import logging

import boto3
import json

# Create a Boto3 client for Bedrock Runtime
bedrock_client = boto3.client(service_name='bedrock-runtime')

max_tokens_to_sample = 200

# Define the prompt and other parameters
prompt = f"""\n\nHuman: 
Write a long and high-quality story about two dogs. Make the story longer than {max_tokens_to_sample}

Rex and Charlie were best friends who did everything together. They lived next door to each other with their human families and spent all day playing in the backyard. Rex was a golden retriever, always happy and eager for fun. Charlie was a German shepherd, more serious but very loyal. 

Every morning, Rex and Charlie would wake up and bark excitedly, ready to start the day's adventures. Their families would let them out into the backyard and they'd run around chasing each other and sniffing for interesting smells. After tiring themselves out, they'd nap in the shade of the big oak tree, Rex's tail still thumping contentedly even in his sleep. 

\n\nAssistant:"""



modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

body = json.dumps({
    "prompt": prompt,
    # specify the parameters as needed
    "max_tokens_to_sample": max_tokens_to_sample,
    "temperature": 0.1,
    "top_p": 0.9,
})

class LLMUser(User):
    @task
    def generation(self):
        # Invoke the model
        with self.environment.events.request.measure("[Send]", "Prompt"):
            response = bedrock_client.invoke_model(
                modelId=modelId,
                accept=accept,
                contentType=contentType,
                body=body
            )
            # Process the response
            response_body = json.loads(response.get('body').read())
            logging.info(response_body.get('completion'))
            
        logging.info("Finished generation!")            


Overwriting locustfile.py


The configuration with Command Line Options https://docs.locust.io/en/stable/configuration.html

--users <int> Peak number of concurrent Locust users. Primarily used together with --headless or --autostart.
    
--headless Disable the web interface, and start the test immediately.
    
--csv Store request stats to files in CSV format.

--spawn-rate <float> Rate to spawn users at (users per second)

In this example, the --users option sets the total number of users to 30, and the --spawn-rate option sets the rate of user spawning to 30 users per second. By using the same value for --spawn-rate as the total number of users, all 30 users will be spawned immediately. Therefore, at any given time during the test, there will be a maximum of 30 concurrent users.

Please note that the --run-time option sets the duration of the test in seconds. In this example, the test will run for 120 seconds before stopping.

!locust --headless --users 10 --spawn-rate 10 --run-time 120 --csv ./benchmark_metric/benchmark_u30

In [69]:
!locust --headless --users 30 --spawn-rate 30 --run-time 120 --csv ./benchmark_metric/benchmark_u30

[2024-02-26 07:08:17,728] ip-172-16-188-233.ec2.internal/INFO/locust.main: Run time limit set to 120 seconds
[2024-02-26 07:08:17,728] ip-172-16-188-233.ec2.internal/INFO/locust.main: Starting Locust 2.23.1
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated       0     0(0.00%) |      0       0       0      0 |    0.00        0.00

[2024-02-26 07:08:17,729] ip-172-16-188-233.ec2.internal/INFO/locust.runners: Ramping to 30 users at a rate of 30.00 per second
[2024-02-26 07:08:17,730] ip-172-16-188-233.ec2.internal/INFO/locust.runners: All users spawned: {"LLMUser": 30} (30 total users)
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
------